<a href="https://colab.research.google.com/github/haesookimDev/MLOps/blob/main/Study/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# fullstackdeeplearning.com 에서 제공한 강의를 기반으로 실습한 코드입니다.


일부 기능 구현 코드에 오류가 있어 수정한 부분이 있습니다.

실습 세팅

In [1]:
lab_idx = 1

if "bootstrap" not in locals() or bootstrap.run:
    # path management for Python
    pythonpath, = !echo $PYTHONPATH
    if "." not in pythonpath.split(":"):
        pythonpath = ".:" + pythonpath
        %env PYTHONPATH={pythonpath}
        !echo $PYTHONPATH

    # get both Colab and local notebooks into the same state
    !wget --quiet https://fsdl.me/gist-bootstrap -O bootstrap.py
    import bootstrap

    # change into the lab directory
    bootstrap.change_to_lab_dir(lab_idx=lab_idx)

    # allow "hot-reloading" of modules
    %load_ext autoreload
    %autoreload 2
    # needed for inline plots in some contexts
    %matplotlib inline

    bootstrap.run = False  # change to True re-run setup

!pwd
%ls

env: PYTHONPATH=.:/env/python
.:/env/python
/content/fsdl-text-recognizer-2022-labs/lab01
notebooks/  text_recognizer/


MNIS 이미지 데이터 셋(숫자 필기체) 로드

In [2]:
from pathlib import Path
import requests


def download_mnist(path):
    url = "https://github.com/pytorch/tutorials/raw/main/_static/"
    filename = "mnist.pkl.gz"

    if not (path / filename).exists():
        content = requests.get(url + filename).content
        (path / filename).open("wb").write(content)

    return path / filename


data_path = Path("data") if Path("data").exists() else Path("../data")
path = data_path / "downloaded" / "vector-mnist"
path.mkdir(parents=True, exist_ok=True)

datafile = download_mnist(path)

다운로드한 데이터 읽어서 훈련, 검증데이터로 분리

In [3]:
import gzip
import pickle


def read_mnist(path):
    with gzip.open(path, "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_valid, y_valid = read_mnist(datafile)

읽어온 데이터를 텐서형태로 변환 > 텐서는 GPU, TPU와 같은 가속기 메모리에서 사용 가능

In [4]:
import torch


x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid)
)

In [5]:
print(x_train, y_train, sep="\n")

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])


In [6]:
y_train[0], x_train[0, ::2]

(tensor(5),
 tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0117, 0.0703, 0.4922, 0.6836, 0.6484,
         0.9648, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1172, 0.3672,
         0.6641, 0.9883, 0.9883, 0.8789, 0.9883, 0.7617, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.9297, 0.9883, 0.9883, 0.9883, 0.9883, 0.3633,
         0.3203,

In [7]:
x_train.ndim, y_train.ndim

(2, 1)

In [8]:
x_train[0, 0], y_train[0]

(tensor(0.), tensor(5))

In [9]:
n, c = x_train.shape
print(x_train.shape)
print(y_train.shape)

torch.Size([50000, 784])
torch.Size([50000])


In [10]:
import random

import wandb  # Tracking, 시각화할 수 있는 Tool

import text_recognizer.metadata.mnist as metadata # mnist 데이터로 부터 추출한 metadata

idx = random.randint(0, len(x_train))
example = x_train[idx]

print(y_train[idx])  # 이미지 정답
wandb.Image(example.reshape(*metadata.DIMS)).image  # 이미지 그리기

tensor(4)


In [11]:
import math

import torch

#실습을 위한 단일 레이어의 가중치와 bias를 초기화 Grediant Descent를 통한 가중치 수정을 위한 requires_grad 옵션 활성화
weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [12]:
#선형 회귀 모델 레이어 선언 입력과 return 타입은 torch.Tensor
# @는 행렬연산자
def linear(x: torch.Tensor) -> torch.Tensor:
    return x @ weights + bias

In [114]:
#선형 회귀 모델의 출력에 softmax값을 취함
def softmax(x: torch.Tensor, dim: int=0) -> torch.Tensor:
    exp_x=torch.exp(x)
    return exp_x/torch.sum(exp_x, axis=1)[:, None]

def model(xb: torch.Tensor) -> torch.Tensor:
    return softmax(linear(xb))

In [115]:
bs = 64  # 배치 사이즈

xb = x_train[0:bs]  # 입력의 첫번째 배치
outs = model(xb)  # 첫번째 배치 결과

print(outs[0], outs.shape)  # 출력

tensor([0.0627, 0.0960, 0.2045, 0.0923, 0.0921, 0.1048, 0.1303, 0.0384, 0.1038,
        0.0750], grad_fn=<SelectBackward0>) torch.Size([64, 10])


In [116]:
#정확도 계산 전체 예측결과중 제대로 예측한 비율
def accuracy(out: torch.Tensor, yb: torch.Tensor) -> torch.Tensor:
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [117]:
yb = y_train[0:bs]

acc = accuracy(outs, yb)

print(acc)

tensor(0.0312)


In [118]:
#정확도 수치 기반으로 역전파
try:
    acc.backward()
except RuntimeError as e:
    print(e)

element 0 of tensors does not require grad and does not have a grad_fn


In [120]:
outs.shape[0]

64

In [128]:
yb

tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5,
        9, 3, 3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 4, 4, 6, 0])

In [129]:
#크로스 엔트로피 실제 값과 예측값 사이의 교차 엔트로피 계산

def cross_entropy(output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    return -torch.log(output[range(target.shape[0]), target]).mean()

loss_func = cross_entropy

In [130]:
print(loss_func(outs, yb), -torch.log(torch.tensor(1 / 10)))

tensor(2.3170, grad_fn=<NegBackward0>) tensor(2.3026)


In [131]:
loss = loss_func(outs, yb)

loss.backward()

In [132]:
bias.grad

tensor([-0.0081, -0.0146,  0.0947, -0.0175, -0.0381,  0.0247,  0.0126, -0.0299,
         0.0065, -0.0303])

In [133]:
lr = 0.5  # 학습률
epochs = 2  # 훈련 에포크

for epoch in range(epochs):  # 에포크 반복
    for ii in range((n - 1) // bs + 1):  # 배치사이즈 만큼 분할
        start_idx = ii * bs  # 배치의 시작 인덱스
        end_idx = start_idx + bs  # 배치 마지막 인덱스

        # 배치만큼 데이터 분할
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]

        # 모델 실행
        pred = model(xb)

        # 손실값 추출
        loss = loss_func(pred, yb)

        # 역전파 과정
        loss.backward()

        # 파라미터 업데이트
        with torch.no_grad():  # no_grad() with statement에 포함해 autograd를 끔으로써 메모리 사용량을 줄이고 연산 속도를 높히기 위함
            # SGD learning rule: update with negative gradient scaled by lr
            weights -= weights.grad * lr
            bias -= bias.grad * lr

            # gradient를 계산할 때 pytorch가 자동으로 gradient 값을 누적
            # .backward()이후 각 파라미터의 .grad에 누적, 나중 .backward()에 영향을 끼침
            # .grad에 원래 누적된 값을 치워주고 (=0으로 초기화 해주고) gradient를 계산
            # .grad.zero_()는 파라미터에 적용 weight .grad.zero_(), bias.grad.zero_()
            #  zero_grad()는 옵티마이저에 적용 모든 파라미터에 .grad.zero_()를 적용 불편, optimizer를 처음에 부를 때 어떤 파라미터를 최적화하고 싶은지 정의
            weights.grad.zero_()
            bias.grad.zero_()

In [134]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))

tensor(0.0805, grad_fn=<NegBackward0>) tensor(1.)


In [135]:
idx = random.randint(0, len(x_train))
example = x_train[idx:idx+1]

out = model(example)

print(out.argmax())
wandb.Image(example.reshape(28, 28)).image

tensor(1)


## 위에 구현한 것들을 pytorch에서 구현된 라이브러리를 호출해 사용

In [136]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

In [137]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb)) #위에서 나온 결과와 동일함

tensor(0.0805, grad_fn=<NllLossBackward0>) tensor(1.)


In [138]:
from torch import nn


class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()  # 클래스를 선언할때 초기화되는 부분
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

In [140]:
def forward(self, xb: torch.Tensor) -> torch.Tensor:# 모델 훈련, 예측 수행할때 실핸하는 부분
    return xb @ self.weights + self.bias

In [141]:
MNISTLogistic.forward = forward #정의한 forward 함수를 오버라이딩

model = MNISTLogistic()  # 모듈 클래스 초기화
print(model(xb)[:4])

loss = loss_func(model(xb), yb)
loss.backward()

print(model.weights.grad[::17,::2])

tensor([[-0.1180,  0.5516, -0.7858, -0.1832, -0.0513,  0.2706,  0.0476,  0.0681,
         -0.5595, -0.1869],
        [ 0.1686,  0.3934, -0.6926, -0.4720, -0.1936,  0.5853, -0.0631, -0.0033,
         -0.4334, -0.0016],
        [-0.0560,  0.3204, -0.5259, -0.0075, -0.0679,  0.1063, -0.0378,  0.0783,
         -0.4344, -0.2066],
        [-0.1292,  0.1544, -0.5223, -0.2212,  0.0370,  0.2705,  0.0609,  0.0114,
          0.1091, -0.1857]], grad_fn=<SliceBackward0>)
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 6.9874e-03,  5.4888e-03,  5.6481e-03,  6.4327e-03,  5.9681e-03],
        [ 0.

In [142]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[-0.0618,  0.0410,  0.0120,  ..., -0.0034,  0.0596, -0.0061],
        [-0.0083, -0.0227,  0.0369,  ..., -0.0205,  0.0271, -0.0072],
        [-0.0182, -0.0017, -0.0283,  ..., -0.0311,  0.0007, -0.0333],
        ...,
        [-0.0360, -0.0289,  0.0415,  ..., -0.0212,  0.0080,  0.0649],
        [ 0.0218, -0.0030,  0.0368,  ..., -0.0038, -0.0193, -0.0091],
        [ 0.0247, -0.0573,  0.0454,  ...,  0.0955,  0.0307,  0.0241]],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)


In [143]:
def fit():
    for epoch in range(epochs):
        for ii in range((n - 1) // bs + 1):
            start_idx = ii * bs
            end_idx = start_idx + bs
            xb = x_train[start_idx:end_idx]
            yb = y_train[start_idx:end_idx]
            pred = model(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            with torch.no_grad():
                for p in model.parameters():  # 모델의 파라미터 업데이트
                    p -= p.grad * lr
                model.zero_grad()

fit()

In [144]:
print(accuracy(model(xb), yb))

tensor(1.)


## 파이토치에 구현된 레이어 사용

In [145]:
import textwrap

print("torch.nn.Modules:", *textwrap.wrap(", ".join(torch.nn.modules.__all__)), sep="\n\t")

torch.nn.Modules:
	Module, Identity, Linear, Conv1d, Conv2d, Conv3d, ConvTranspose1d,
	ConvTranspose2d, ConvTranspose3d, Threshold, ReLU, Hardtanh, ReLU6,
	Sigmoid, Tanh, Softmax, Softmax2d, LogSoftmax, ELU, SELU, CELU, GLU,
	GELU, Hardshrink, LeakyReLU, LogSigmoid, Softplus, Softshrink,
	MultiheadAttention, PReLU, Softsign, Softmin, Tanhshrink, RReLU,
	L1Loss, NLLLoss, KLDivLoss, MSELoss, BCELoss, BCEWithLogitsLoss,
	NLLLoss2d, PoissonNLLLoss, CosineEmbeddingLoss, CTCLoss,
	HingeEmbeddingLoss, MarginRankingLoss, MultiLabelMarginLoss,
	MultiLabelSoftMarginLoss, MultiMarginLoss, SmoothL1Loss,
	GaussianNLLLoss, HuberLoss, SoftMarginLoss, CrossEntropyLoss,
	Container, Sequential, ModuleList, ModuleDict, ParameterList,
	ParameterDict, AvgPool1d, AvgPool2d, AvgPool3d, MaxPool1d, MaxPool2d,
	MaxPool3d, MaxUnpool1d, MaxUnpool2d, MaxUnpool3d, FractionalMaxPool2d,
	FractionalMaxPool3d, LPPool1d, LPPool2d, LocalResponseNorm,
	BatchNorm1d, BatchNorm2d, BatchNorm3d, InstanceNorm1d, InstanceNorm2d,

In [146]:
class MNISTLogistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = nn.Linear(784, 10)  # 입력 크기 784 출력 10 (클래스 수)

    def forward(self, xb):
        return self.lin(xb)  # 선형모델 선언

In [147]:
model = MNISTLogistic()

print(loss_func(model(xb), yb))  # loss is still close to 2.3

tensor(2.3510, grad_fn=<NllLossBackward0>)


In [148]:
print(*list(model.children()))

Linear(in_features=784, out_features=10, bias=True)


In [149]:
print(*list(model.parameters()), sep="\n")

Parameter containing:
tensor([[ 0.0030, -0.0112, -0.0256,  ..., -0.0068,  0.0033,  0.0197],
        [-0.0220, -0.0283,  0.0276,  ...,  0.0117, -0.0098, -0.0109],
        [-0.0229,  0.0015,  0.0125,  ...,  0.0019, -0.0232, -0.0326],
        ...,
        [-0.0331, -0.0261,  0.0340,  ..., -0.0104, -0.0174,  0.0219],
        [ 0.0081,  0.0263,  0.0020,  ..., -0.0267,  0.0031, -0.0031],
        [-0.0073,  0.0156,  0.0299,  ...,  0.0250,  0.0042,  0.0044]],
       requires_grad=True)
Parameter containing:
tensor([ 0.0166,  0.0085,  0.0179,  0.0009, -0.0122,  0.0345, -0.0011, -0.0182,
         0.0037,  0.0023], requires_grad=True)


In [150]:
from torch import optim

#기존에 구현한 옵티마이저는 SGD를 이용했으나 Adam이 더 성능이 좋음
def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)

In [151]:
model = MNISTLogistic()
opt = configure_optimizer(model)

print("before training:", loss_func(model(xb), yb), sep="\n\t")

for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        start_idx = ii * bs
        end_idx = start_idx + bs
        xb = x_train[start_idx:end_idx]
        yb = y_train[start_idx:end_idx]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print("after training:", loss_func(model(xb), yb), sep="\n\t")

before training:
	tensor(2.3173, grad_fn=<NllLossBackward0>)
after training:
	tensor(0.8501, grad_fn=<NllLossBackward0>)


## 새로운 데이터

In [152]:
from text_recognizer.data.util import BaseDataset


train_ds = BaseDataset(x_train, y_train)

In [153]:
model = MNISTLogistic()
opt = configure_optimizer(model)


for epoch in range(epochs):
    for ii in range((n - 1) // bs + 1):
        xb, yb = train_ds[ii * bs: ii * bs + bs]  # xb and yb in one line!
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        opt.step()
        opt.zero_grad()

print(loss_func(model(xb), yb))

tensor(0.8564, grad_fn=<NllLossBackward0>)


In [154]:
from torch.utils.data import DataLoader


train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [161]:
BaseDataset??

In [155]:
def fit(self: nn.Module, train_dataloader: DataLoader):
    opt = configure_optimizer(self)

    for epoch in range(epochs):
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

MNISTLogistic.fit = fit

In [156]:
model = MNISTLogistic()

model.fit(train_dataloader)

print(loss_func(model(xb), yb))

tensor(0.8562, grad_fn=<NllLossBackward0>)


In [157]:
from text_recognizer.models.mlp import MLP


MLP.fit = fit  # attach our fitting loop

In [158]:
MLP.forward??

In [159]:
MLP.__init__??

In [160]:
digits_to_9 = list(range(10))
data_config = {"input_dims": (784,), "mapping": {digit: str(digit) for digit in digits_to_9}}
data_config

{'input_dims': (784,),
 'mapping': {0: '0',
  1: '1',
  2: '2',
  3: '3',
  4: '4',
  5: '5',
  6: '6',
  7: '7',
  8: '8',
  9: '9'}}

In [162]:
model = MLP(data_config)
model

MLP(
  (fc1): Linear(in_features=784, out_features=1024, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [163]:
model.fc1.weight

Parameter containing:
tensor([[ 0.0272, -0.0230,  0.0161,  ...,  0.0141, -0.0187, -0.0055],
        [-0.0314,  0.0281, -0.0129,  ..., -0.0020,  0.0331,  0.0247],
        [-0.0142, -0.0088, -0.0219,  ...,  0.0343, -0.0313, -0.0311],
        ...,
        [ 0.0218, -0.0122, -0.0005,  ..., -0.0196,  0.0139,  0.0143],
        [ 0.0343, -0.0233,  0.0190,  ...,  0.0164, -0.0012, -0.0270],
        [ 0.0045,  0.0095, -0.0269,  ...,  0.0076,  0.0275,  0.0043]],
       requires_grad=True)

In [164]:
%%time

print("before training:", loss_func(model(xb), yb))

train_ds = BaseDataset(x_train, y_train)
train_dataloader = DataLoader(train_ds, batch_size=bs)
fit(model, train_dataloader)

print("after training:", loss_func(model(xb), yb))

before training: tensor(2.3059, grad_fn=<NllLossBackward0>)
after training: tensor(0.3664, grad_fn=<NllLossBackward0>)
CPU times: user 28.6 s, sys: 44.4 ms, total: 28.6 s
Wall time: 28.9 s


In [165]:
if torch.cuda.is_available():
    !nvidia-smi
else:
    print("☹️")

☹️


In [166]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

loss_func(model(xb.to(device)), yb.to(device))

tensor(0.1620, grad_fn=<NllLossBackward0>)

In [167]:
def push_to_device(tensor):
    return tensor.to(device)

train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
train_dataloader = DataLoader(train_ds, batch_size=bs)

In [168]:
%%time

model = MLP(data_config)
model.to(device)

model.fit(train_dataloader)

print(loss_func(model(push_to_device(xb)), push_to_device(yb)))

tensor(0.2254, grad_fn=<NllLossBackward0>)
CPU times: user 28.4 s, sys: 21.8 ms, total: 28.4 s
Wall time: 28.6 s


In [169]:
class MNISTDataModule:
    url = "https://github.com/pytorch/tutorials/raw/master/_static/"
    filename = "mnist.pkl.gz"

    def __init__(self, dir, bs=32):
        self.dir = dir
        self.bs = bs
        self.path = self.dir / self.filename

    def prepare_data(self):
        if not (self.path).exists():
            content = requests.get(self.url + self.filename).content
            self.path.open("wb").write(content)

    def setup(self):
        with gzip.open(self.path, "rb") as f:
            ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

        x_train, y_train, x_valid, y_valid = map(
            torch.tensor, (x_train, y_train, x_valid, y_valid)
            )

        self.train_ds = BaseDataset(x_train, y_train, transform=push_to_device, target_transform=push_to_device)
        self.valid_ds = BaseDataset(x_valid, y_valid, transform=push_to_device, target_transform=push_to_device)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds, batch_size=self.bs, shuffle=True)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.valid_ds, batch_size=2 * self.bs, shuffle=False)

In [170]:
def fit(self: nn.Module, datamodule):
    datamodule.prepare_data()
    datamodule.setup()

    val_dataloader = datamodule.val_dataloader()

    self.eval()
    with torch.no_grad():
        valid_loss = sum(loss_func(self(xb), yb) for xb, yb in val_dataloader)

    print("before start of training:", valid_loss / len(val_dataloader))

    opt = configure_optimizer(self)
    train_dataloader = datamodule.train_dataloader()
    for epoch in range(epochs):
        self.train()
        for xb, yb in train_dataloader:
            pred = self(xb)
            loss = loss_func(pred, yb)

            loss.backward()
            opt.step()
            opt.zero_grad()

        self.eval()
        with torch.no_grad():
            valid_loss = sum(loss_func(self(xb), yb) for xb, yb in val_dataloader)

        print(epoch, valid_loss / len(val_dataloader))


MNISTLogistic.fit = fit
MLP.fit = fit

In [171]:
model = MLP(data_config)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=32)

model.fit(datamodule=datamodule)

before start of training: tensor(2.3043)
0 tensor(0.1672)
1 tensor(0.1130)


In [172]:
MLP.__init__??

In [173]:
%%time
from argparse import Namespace  # you'll need this

args = None  # edit this

epochs = 2  # used in fit
bs = 32  # used by the DataModule


# used in fit, play around with this if you'd like
def configure_optimizer(model: nn.Module) -> optim.Optimizer:
    return optim.Adam(model.parameters(), lr=3e-4)


model = MLP(data_config, args=args)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=bs)

model.fit(datamodule=datamodule)

before start of training: tensor(2.3069)
0 tensor(0.1686)
1 tensor(0.1156)
CPU times: user 54.7 s, sys: 847 ms, total: 55.6 s
Wall time: 55.9 s


In [174]:
val_dataloader = datamodule.val_dataloader()
valid_acc = sum(accuracy(model(xb), yb) for xb, yb in val_dataloader) / len(val_dataloader)
valid_acc

tensor(0.9656)

In [175]:
MLP.forward??

# 실습

In [189]:
import numpy as np
from argparse import Namespace

In [198]:
class YourModel(nn.Module):
    def __init__(
        self,
        data_config: dict[str, any],
        args: Namespace = None,
    ) -> None:  # add args and kwargs here as you like

        super().__init__()
        # use those args and kwargs to set up the submodules
        self.args = vars(args) if args is not None else {}
        self.data_config = data_config

        input_dim = np.prod(self.data_config["input_dims"])
        num_classes = len(self.data_config["mapping"])

        self.fc1 = nn.Linear(input_dim, 500)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(500, 200)
        self.bn = nn.BatchNorm1d(200)
        self.fc3 = nn.Linear(200, num_classes)

    def forward(self, x):  # overwrite this to use your nn.Modules from above
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x


YourModel.fit = fit  # don't forget this!

In [199]:
args = None

model = YourModel(data_config, args=args)
model.to(device)

datamodule = MNISTDataModule(dir=path, bs=bs)

model.fit(datamodule=datamodule)

before start of training: tensor(2.3007)
0 tensor(0.1815)
1 tensor(0.1234)


In [200]:
val_dataloader = datamodule.val_dataloader()
valid_acc = sum(accuracy(model(xb), yb) for xb, yb in val_dataloader) / len(val_dataloader)
valid_acc

tensor(0.9647)